- Transformer 是现代人工智能的核心，而attention是 Transformer 最具特色的机制，在 transformer 及 attention 上花再多的时间探索都是值得。
- 从加速计算/存储的角度优化 attention 计算，都是在 attention 的计算机制上做文章：
    - vllm 中的 paged attention；
    - 以及今天要讲的 flash attention（考虑到硬件的读取和计算）；

In [18]:
from IPython.display import Image
import numpy as np
import torch


https://github.com/Dao-AILab/flash-attention

- install
    
    ```
    # pip
    pip install flash-attn --no-build-isolation
    pip install flash_attn -U --force-reinstall
    
    # source code compile
    python setup.py install
    ```
- references
    - paper
        - [FlashAttention: Fast and Memory-Efficient Exact Attention with IO-Awareness](https://arxiv.org/pdf/2205.14135)
        - [FlashAttention-2: Faster Attention with Better Parallelism and Work Partitioning](https://tridao.me/publications/flash2/flash2.pdf)
    - https://medium.com/@e0928021388/%E7%AA%81%E7%A0%B4-transformers-%E7%9A%84%E9%80%9F%E5%BA%A6%E7%93%B6%E9%A0%B8-flash-attention-%E4%BB%8B%E7%B4%B9-28c1bc667fd9

## basics

> 算法（软件）、硬件协同优化；

- Memory
    - SRAM > HBM > DRAM
    - SRAM：Static RAM（Random Access Memory）
        - 每个 SM（Stream multiproecssors，流多处理器）192KB （A100 108个，4090 128个）
            - 108*192/1024 = 20MB
    - HBM：high bandwidth memory，4090:24GB
- GPU 读写&计算（compute-bound vs. memory-bound)
    - operations fused：将好几个 operations fuse 成一个 operation 进而减轻 memory 存取的 loading
    - 在GPU当中有非常大量的 threads （kernel） 负责执行 operation 的运算，而整个运算的过程基本上是从 HBM 当中将资料加载至 SRAM 中，执行运算并将 output 存回 HBM 当中。
    - compute-bound
        - 运算的主要时间都耗费在 operation 的计算上，HBM 的存取只占了其中一点点的时间
        - 像是多维度的矩阵相乘或是高 channel 数的 convolution 都属于这类。
    - memory-bound
        - 主要时间都耗费在 memory 的读取上，而实际的运算只占了其中一点点的时间
        - elementwise （e.g.， activation， dropout） and reduction （e.g.， sum， softmax， batch norm， layer norm）
- attention QKV 计算
    - 分块矩阵，然后是 loop（outer loop，inner loop，对应的是 gpu cuda 的 kernel 优化）；

### review of Attention

In [14]:
Image(url='./imgs/attention_steps.png', width=600)

$$
\mathbf{O} = \text{Dropout}(\text{Softmax}(\text{Mask}(\mathbf{QK}^T)))\mathbf{V}
$$

- formula
    - $(\mathbf {Q, K, V})\in\mathbb R^{N\times d}$
    - $\mathbf{QK}^T\in\mathbb R^{N\times N}$
    - $\mathbf A=\text{Dropout}(\text{Softmax}(\text{Mask}(\mathbf{QK}^T)))$
        - mask: m, softmax: sm, dropout: do
    - $\mathbf O=\mathbf {AV}\in \mathbb R^{N\times N}$
- notes
    - attention is bottlenecked by memory reads/writes
    - naive implementation requires repeated R/W from slow GPU HBM

In [16]:
Image(url='./imgs/stand_attn.png', width=500)

-  memory access 的时间复杂度为 $O（N*D + N*N）$，其中通常 N >> D（e.g.， N 为 4096 而 d 为 64），因此我们可以发现 S 和 P 的 memory access （$N*N$ 的复杂度） 便是整体 self-attention 的 bottleneck！
- 我们可以发现对于整个 self-attention 当中，其实我们真正需要的是最后面的 output O 而已，过程当中不管 P 和 S 长什么样子其实对于我们来说都没有很重要，既然他不重要为什么我们还是要将他存入 HBM 呢？ 主要是因为以下两个理由：
    - 我们需要这些 intermediate activations 来帮助我们在 backward 的时候通过 backpropagation 计算 gradients，这也使得我们很难将多个 operations fuse 成一个 operation。
    - 由于 SRAM 本身不够大，而 softmax 这种需要计算 sum 的 operation，需要整个 row 的 element 都到齐后才可以计算，使得我们沒有办法 apply 一些 divide and conquery 的 algorithm ，更使得我们没有办法把所有运算一口气在 SRAM当中计算完。

## flash attention (Tiling & Recomputation)

In [10]:
Image(url='./imgs/flash-attn.png', width=700)

In [17]:
Image(url='./imgs/stand_attn.png', width=500)

### tiling

- `(Q, K) => S => P => O` => `(Q, K) => O`

- softmax of vector $x\in \mathbb R^B$（element-wise）

    $$
    m(x):=\max_i x_i, \quad f(x):=[e^{x_1-m(x)}, \cdots, e^{x_B-m(x)}], \quad \ell(x)=\sum f(x), \quad \text{softmax}(x):=\frac{f(x)}{\ell(x)}
    $$

- vectors $x^{(1)},x^{(2)}\in \mathbb R^B, x=[x^{(1)}\quad x^{(2)}]\in \mathbb R^{2B}$（element-wise）

    $$
    \begin{split}
    m(x) &= m\left( \left[ x^{(1)}, x^{(2)} \right] \right) = \max \left( m(x^{(1)}), m(x^{(2)}) \right), \\
    f(x) &= \left[ e^{m(x^{(1)}) - m(x)} f(x^{(1)}), \, e^{m(x^{(2)}) - m(x)} f(x^{(2)}) \right], \\
    \ell(x) &= \ell\left( \left[ x^{(1)}, x^{(2)} \right] \right) = e^{m(x^{(1)}) - m(x)} \ell(x^{(1)}) + e^{m(x^{(2)}) - m(x)} \ell(x^{(2)}), \\
    \text{softmax}(x) &= \frac{f(x)}{\ell(x)}.
    \end{split}
    $$
  - 这里的 $f(x^{(1)})=[e^{x_1-m(x^{(1)})}, \cdots]$

In [64]:
x = torch.tensor([1, 2, 3, 4], dtype=torch.float)
x

tensor([1., 2., 3., 4.])

In [62]:
torch.softmax(x, dim=-1)

tensor([0.0321, 0.0871, 0.2369, 0.6439])

In [63]:
m = torch.max(x)
f = torch.exp(x - m)
l = torch.sum(f)
f / l

tensor([0.0321, 0.0871, 0.2369, 0.6439])

In [67]:
x_1 = x[:2]
x_2 = x[2:]
m = torch.max(x)
m_1 = torch.max(x_1)
m_2 = torch.max(x_2)

f_1 = torch.exp(x_1 - m_1)
f_2 = torch.exp(x_2 - m_2)

l_1 = torch.sum(f_1)
l_2 = torch.sum(f_2)

f = torch.cat((torch.exp(m_1 - m) * f_1, torch.exp(m_2 - m) * f_2))
l = torch.exp(m_1 - m) * l_1 + torch.exp(m_2 - m) * l_2
f/l

tensor([0.0321, 0.0871, 0.2369, 0.6439])

In [54]:
Image(url='./imgs/flash-attn-algo.png', width=600)

- $\mathbf {Q,K,V,O}$ 分别做行分块（row blocks），$\mathbf O$ 是结果矩阵
    - $\mathbf Q_i,\mathbf O_i\in \mathbb R^{B_r\times d}$
    - $\mathbf {K}_j,\mathbf {V}_j\in\mathbb R^{B_c\times d}$
- 对 $T_c$（$\mathbf K_j,\mathbf V_j$） 做外循环，对 $T_r$（$\mathbf Q_i, \mathbf O_i$）做内循环
    - 内循环不断地 update $\mathbf O_i$

In [48]:
s = torch.tensor([0.1, 0.3, 0.5, 0.7])
v = torch.tensor([7, 8,  9, 10], dtype=torch.float)

In [26]:
p = torch.softmax(s, dim=-1)
p

tensor([0.1807, 0.2207, 0.2695, 0.3292])

In [27]:
p @ v

tensor(8.7472)

In [39]:
# tiling
s = s.view(2, 2)
s

tensor([[0.1000, 0.3000],
        [0.5000, 0.7000]])

In [42]:
p = torch.softmax(s, dim=-1)
p

tensor([[0.4502, 0.5498],
        [0.4502, 0.5498]])

In [49]:
v = torch.transpose(v.view(2, 2), 0, 1)
v

tensor([[ 7.,  9.],
        [ 8., 10.]])

In [50]:
p[0, :] @ v[:, 0]

tensor(7.5498)

In [51]:
p[1, :] @ v[:, 1]

tensor(9.5498)

In [45]:
# exponential summation (softmax 的分母)
torch.sum(torch.exp(s), dim=-1)

tensor([2.4550, 3.6625])

In [52]:
 (9.5498 * 3.66 + 7.5498 * 2.455) / (2.455 + 3.66)

8.74685641864268

### SDPA（pytorch）

## simple demo

In [5]:
import numpy as np

# 输入矩阵
X = np.array([[1, 2, 3, 4],
              [5, 6, 7, 8],
              [9, 10, 11, 12],
              [13, 14, 15, 16]])

# 权重矩阵
W_Q = W_K = W_V = np.eye(4)

# 经典自注意力机制
Q = np.dot(X, W_Q)
K = np.dot(X, W_K)
V = np.dot(X, W_V)
attention_scores = np.dot(Q, K.T) / np.sqrt(4)
attention_weights = np.exp(attention_scores) / np.sum(np.exp(attention_scores), axis=1, keepdims=True)
output_classic = np.dot(attention_weights, V)

output_classic

array([[12.99999999, 13.99999999, 14.99999999, 15.99999999],
       [13.        , 14.        , 15.        , 16.        ],
       [13.        , 14.        , 15.        , 16.        ],
       [13.        , 14.        , 15.        , 16.        ]])

In [6]:
# Flash Attention
b = 2
m = 2
output_flash = np.zeros((4, 4))
for i in range(b):
    X_block = X[i * m: (i + 1) * m]
    Q_block = np.dot(X_block, W_Q)
    K_block = np.dot(X_block, W_K)
    V_block = np.dot(X_block, W_V)
    
    # 计算块间的注意力得分
    attention_scores_block = np.dot(Q_block, K.T) / np.sqrt(4)
    attention_weights_block = np.exp(attention_scores_block) / np.sum(np.exp(attention_scores_block), axis=1, keepdims=True)
    
    # 累加到输出
    output_flash[i * m: (i + 1) * m] = np.dot(attention_weights_block, V)

output_flash

array([[12.99999999, 13.99999999, 14.99999999, 15.99999999],
       [13.        , 14.        , 15.        , 16.        ],
       [13.        , 14.        , 15.        , 16.        ],
       [13.        , 14.        , 15.        , 16.        ]])